# PyPSRP 最新版本中的一些示例

> PS: 最新版本为 2022.2.22 发布的 0.8.1 版本 - [PyPSRP 0.8.1](https://pypi.org/project/pypsrp/)

In [3]:
# 使用 high level client api
from pypsrp.client import Client
from config import SERVER, USERNAME, PASSWORD
from pathlib import Path


# this takes in the same kwargs as the WSMan object
with Client(
    server=SERVER,
    username=USERNAME,
    password=PASSWORD,
    ssl=False,
) as client:
    # execute a cmd command
    stdout, stderr, rc = client.execute_cmd("dir", encoding="gbk")
    print(f"stdout: {stdout} \nstderr: {stderr} \nrc: {rc}")

    stdout, stderr, rc = client.execute_cmd("powershell.exe gci $pwd", encoding="gbk")
    print(f"stdout: {stdout} \nstderr: {stderr} \nrc: {rc}")
    sanitised_stderr = client.sanitise_clixml(stderr)
    print(f"sanitised stderr: {sanitised_stderr}")

    # execute a PowerShell script - 当前目录下的 test_path.ps1
    script_path = "test_path.ps1"
    with open(script_path, "r") as f:
        script = f.read()
    output, streams, had_errors = client.execute_ps(script)
    print(f"output: {output} \nstreams: {streams} \nhad_errors: {had_errors}")

    output, streams, had_errors = client.execute_ps(
        "New-Item -Path C:\\temp\\folder -ItemType Directory"
    )
    print(f"output: {output} \nstreams: {streams} \nhad_errors: {had_errors}")

    # copy a file from the local host to the remote host
    client.copy(str(script_path), "C:\\temp\\file.txt")

    # fetch a file from the remote host to the local host
    dst_path = "file.txt"
    client.fetch("C:\\temp\\file.txt", str(dst_path))


stdout:  驱动器 C 中的卷没有标签。
 卷的序列号是 E064-A9DD

 C:\Users\win10pro 的目录

2023/05/25  23:09    <DIR>          .
2023/05/25  23:09    <DIR>          ..
2023/05/25  23:07    <DIR>          3D Objects
2023/05/25  23:07    <DIR>          Contacts
2023/05/26  00:09    <DIR>          Desktop
2023/11/08  23:55    <DIR>          Documents
2023/05/25  23:07    <DIR>          Downloads
2023/05/25  23:07    <DIR>          Favorites
2023/05/25  23:07    <DIR>          Links
2023/05/25  23:07    <DIR>          Music
2023/05/25  23:09    <DIR>          OneDrive
2023/05/25  23:09    <DIR>          Pictures
2023/05/25  23:07    <DIR>          Saved Games
2023/05/25  23:09    <DIR>          Searches
2023/05/25  23:07    <DIR>          Videos
               0 个文件              0 字节
              15 个目录 80,802,869,248 可用字节
 
stderr:  
rc: 0
stdout: 

    目录: C:\Users\win10pro


Mode                 LastWriteTime         Length Name                                                                 
----            

使用 WinRS/Process 执行命令:

In [4]:
from pypsrp.shell import Process, SignalCode, WinRS
from pypsrp.wsman import WSMan
from config import SERVER, USERNAME, PASSWORD


# creates a http connection with no encryption and basic auth
wsman = WSMan(
    server=SERVER,
    username=USERNAME,
    password=PASSWORD,
    ssl=False,
)

with wsman, WinRS(wsman) as shell:
    process = Process(shell, "dir")
    process.invoke()
    process.signal(SignalCode.CTRL_C)
    print(
        f"stdout: {process.stdout.decode('gbk')} \n=======\nstderr: {process.stderr.decode('gbk')} \n======\nrc: {process.rc}\n======\n"
    )

    # execute a process with arguments in the background
    process = Process(shell, "powershell", ["gci", "$pwd"])
    process.begin_invoke()  # start the invocation and return immediately
    process.poll_invoke()  # update the output stream
    process.end_invoke()  # finally wait until the process is finished
    process.signal(SignalCode.CTRL_C)
    print(
        f"stdout: {process.stdout.decode('gbk')} \n=======\nstderr: {process.stderr.decode('gbk')} \n======\nrc: {process.rc}\n======\n"
    )


stdout:  驱动器 C 中的卷没有标签。
 卷的序列号是 E064-A9DD

 C:\Users\win10pro 的目录

2023/05/25  23:09    <DIR>          .
2023/05/25  23:09    <DIR>          ..
2023/05/25  23:07    <DIR>          3D Objects
2023/05/25  23:07    <DIR>          Contacts
2023/05/26  00:09    <DIR>          Desktop
2023/11/08  23:55    <DIR>          Documents
2023/05/25  23:07    <DIR>          Downloads
2023/05/25  23:07    <DIR>          Favorites
2023/05/25  23:07    <DIR>          Links
2023/05/25  23:07    <DIR>          Music
2023/05/25  23:09    <DIR>          OneDrive
2023/05/25  23:09    <DIR>          Pictures
2023/05/25  23:07    <DIR>          Saved Games
2023/05/25  23:09    <DIR>          Searches
2023/05/25  23:07    <DIR>          Videos
               0 个文件              0 字节
              15 个目录 80,802,406,400 可用字节
 
stderr:  
rc: 0

stdout: 

    目录: C:\Users\win10pro


Mode                 LastWriteTime         Length Name                                                                 
----           

使用 RunspacePool/PowerShell 执行 PowerShell 脚本/命令:

In [1]:
from pypsrp.powershell import PowerShell, RunspacePool
from pypsrp.wsman import WSMan
from config import SERVER, USERNAME, PASSWORD


wsman = WSMan(
    server=SERVER,
    username=USERNAME,
    password=PASSWORD,
    ssl=False,
)

with wsman, RunspacePool(wsman) as pool:
    # execute 'Get-Process | Select-Object Name'
    ps = PowerShell(pool)
    ps.add_cmdlet("Get-Process").add_cmdlet("Select-Object").add_argument("Name")
    output = ps.invoke()
    print(f"Output: {output}")

    # execute 'Get-Process | Select-Object -Property Name'
    ps = PowerShell(pool)
    ps.add_cmdlet("Get-Process").add_cmdlet("Select-Object")
    ps.add_parameter("Property", "Name")
    ps.begin_invoke()  # execute process in the background
    ps.poll_invoke()  # update the output streams
    ps.end_invoke()  # wait until the process is finished
    print(f"Output: {ps.output}")

    # execute 'Get-Process | Select-Object -Property Name; Get-Service audiosrv'
    ps = PowerShell(pool)
    ps.add_cmdlet("Get-Process").add_cmdlet("Select-Object").add_parameter(
        "Property", "Name"
    )
    ps.add_statement()
    ps.add_cmdlet("Get-Service").add_argument("audiosrc")
    ps.invoke()
    print(f"Output: {ps.output}")

    # execute a PowerShell script with input being sent
    ps = PowerShell(pool)
    script = """begin {
    $DebugPreference = "Continue"
    Write-Debug -Message "begin"
} process {
    Write-Output -InputObject $input
} end {
    Write-Debug -Message "end"
}
"""
    ps.add_script(script)
    ps.invoke(["string", 1])
    print(ps.output)
    print(ps.streams.debug)


Output: [<pypsrp.complex_objects.GenericComplexObject object at 0x000001579152ED40>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152F4F0>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152EBF0>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152F550>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152EAD0>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152DFC0>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152EAA0>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152F8E0>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152FC40>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152FA60>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152EDA0>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152E3E0>, <pypsrp.complex_objects.GenericComplexObject object at 0x000001579152E7A0>, <py

## Logging

该库利用 Python 日志记录配置，并将消息记录到 pypsrp 命名记录器以及 pypsrp.* ，其中 * 是 pypsrp 目录。

> 最终也没搞定这个 Logging 怎么用
